# Run a Flask App Server Inside Your Colab Notebook
The companion blog article to this notebook can be found here.  http://www.storminthecastle.com/posts/colab_web/

This notebook demonstrates running and embedded flask webservice within your colab notebook allowing you to unlock resources like the colab GPU for more interactive web applications.

For updates and more content follow me on twitter [@johnrobinsn](https://www.twitter.com/johnrobinsn) or on my blog [storminthecastle.com](https://www.storminthecastle.com).

## A Flask Appserver Running in Your Notebook

In [1]:
# Run a python(flask)-based web service in your note book
# You can reload this cell to restart the server if you make changes

default_port = 6060

from werkzeug.serving import make_server
from flask import Flask
import threading

class ServerThread(threading.Thread):

    def __init__(self, app, port):
        threading.Thread.__init__(self)
        self.port = port
        self.srv = make_server('127.0.0.1', port, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print('starting server on port:',self.port)
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

def start_server(port=default_port):
    global server
    if 'server' in globals() and server:
      print('stopping server')
      stop_server()

    app = Flask('myapp')


    # you can add your own routes here as needed
    @app.route("/")
    def hello():
      # A wee bit o'html
      return '<h1 style="color:red;">Hello From Flask!</h1>'

    server = ServerThread(app,port)
    server.start()

def stop_server():
    global server
    if server:
      server.shutdown()
      server = None

# Start the server here
start_server()

starting server on port: 6060


## Demonstrate an HTTP Request from this notebook to the Embedded Flask Server

In [2]:
# Make a HTTP request to our little embedded webservice

import requests

r = requests.get('http://localhost:6060')
print(r.text)

INFO:werkzeug:127.0.0.1 - - [06/May/2025 22:29:53] "GET / HTTP/1.1" 200 -


<h1 style="color:red;">Hello From Flask!</h1>


## Create an Embedded &lt;iframe&gt; and Load Content from Flask.

In [3]:
# By executing this cell you will dynamically create an iframe within this cell of your notebook
# this iframe can load an arbitrary web application which can access the flask webservice that is running
# within this notebook.

import IPython.display

def display(port, height):
    shell = """
        (async () => {
            const url = await google.colab.kernel.proxyPort(%PORT%, {"cache": true});
            const iframe = document.createElement('iframe');
            iframe.src = url;
            iframe.setAttribute('width', '100%');
            iframe.setAttribute('height', '%HEIGHT%');
            iframe.setAttribute('frameborder', 0);
            document.body.appendChild(iframe);
        })();
    """
    replacements = [
        ("%PORT%", "%d" % port),
        ("%HEIGHT%", "%d" % height),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)

    script = IPython.display.Javascript(shell)
    IPython.display.display(script)

display(default_port, 400)

<IPython.core.display.Javascript object>

## In a future blog article I'll show you how to use these tools to add an interactive 3d visualization to your notebook using three.js.  


For updates and more content follow me on twitter [@johnrobinsn](https://www.twitter.com/johnrobinsn) or on my blog [storminthecastle.com](https://www.storminthecastle.com).
